In [1]:
#imports
from flask import Flask, request
from flask_restful import Api, Resource, reqparse,abort, fields, marshal_with
from flask_sqlalchemy import SQLAlchemy



app = Flask(__name__)
api= Api(app)

#making a config. Defining where the database is
app.config['SQLALCHEMY_DATABASE_URI']= 'sqlite:///database.db'
db = SQLAlchemy (app)

#define models in database
class GameModel(db.Model):
    id = db.Column(db.Integer, primary_key=True)#unique Identifier
    name = db.Column(db.String(100),nullable=False)
    genre = db.Column(db.String(100), nullable=False)
    summary = db.Column(db.String, nullable=True) #summary isn't required and also doesn't have the same
    #string limit


    #represent class objects as a string
    def __reper__(self):
        return f"Games(name={name}, type ={genre}, summary={summary})"


#run this once
db.create_all()

#argument parsers
#This is the information I want about every game entered
game_args=reqparse.RequestParser()
game_args.add_argument("name",type=str, help="Name of game", required=True )
game_args.add_argument("genre", type=str, help="Genre of the game ", required=True)
game_args.add_argument("summary", type=str, help= "Summary of the game", required=False)


#defines how an object should be seralized
resource_fields ={
    'id':fields.Integer,
    'name': fields.String,
    'genre': fields.String,
    'summary': fields.String}



#add a resource
class Games(Resource): #a class that is a resource, and will have overridable options (get,put, delete ect.)
    @marshal_with(resource_fields)#seralize
    def get(self, game_id):
        result=GameModel.query.filter_by(id=game_id).first() #this returns an instance of the above class. remember outputs must be seralized 
        if not result:
            abort(404, message="could not find the game ID")
        return result
        
    @marshal_with(resource_fields) 
    def post(self, game_id): 
        args= game_args.parse_args() #gets all arguments
        result=GameModel.query.filter_by(id=game_id).first()
        if result:
            abort(409, message= "ID taken")


        #make object in database
        game = GameModel(id=game_id, name=args['name'], genre=args['genre'], summary=args['summary'])


        #commit to database
        db.session.add(game) #temp add
        db.session.commit() #perm inside. Consider noting this line on tests.
        return game, 201



        
api.add_resource(Games, "/games/<int:game_id>")
#register the above as a resource. 
# "/" refers to default url. 
#however "/games is how this resource is accessed
#<>  define the kind of parameter the user needs to pass after
#For example after the first game entry the link is now https:xyz/games/0

#Homepage
@app.route("/")
def home():
    return "Hi welcome to the homepage. After inputing data using test.py you can navigate to the specific datapage by changing the URL. For example your first Data entry will be found at http://http://127.0.0.1:5000/games/0"


if __name__ == '__main__':
    app.run(debug=True,use_reloader=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


C:\Users\DANY\anaconda3\lib\site-packages\flask_sqlalchemy\__init__.py:833: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [17/Jan/2022 16:01:57] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Jan/2022 16:01:58] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [17/Jan/2022 16:02:05] "GET /games/0 HTTP/1.1" 404 -
